In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Albany,US,42.6001,-73.9662,66.02,80,100,3.15,overcast clouds
1,1,Busselton,AU,-33.6500,115.3333,59.04,63,0,10.76,clear sky
2,2,Evensk,RU,61.9500,159.2333,55.85,75,41,9.66,scattered clouds
3,3,Saint-Pierre,RE,-21.3393,55.4781,65.84,82,75,9.22,broken clouds
4,4,Rikitea,PF,-23.1203,-134.9692,74.82,75,0,12.82,clear sky
5,5,Lagos,NG,6.5833,3.7500,77.32,86,100,5.28,overcast clouds
6,6,Borba,BR,-4.3878,-59.5939,77.68,72,77,2.33,broken clouds
7,7,Yellowknife,CA,62.4560,-114.3525,61.32,67,75,14.97,broken clouds
8,8,Clyde River,CA,70.4692,-68.5914,43.70,79,95,1.05,overcast clouds
9,9,Saldanha,ZA,-33.0117,17.9442,51.85,66,11,11.77,few clouds


In [3]:

# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = int(input('What is the minimum temperature you want in your trip?'))
max_temp = int(input('What is the maximum temperature you want in your trip?'))



What is the minimum temperature you want in your trip?68
What is the maximum temperature you want in your trip?85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & \
                 (city_data_df['Max Temp'] >= min_temp)]
preferred_cities.head(10)



,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
4,4,Rikitea,PF,-23.1203,-134.9692,74.82,75,0,12.82,clear sky
5,5,Lagos,NG,6.5833,3.7500,77.32,86,100,5.28,overcast clouds
6,6,Borba,BR,-4.3878,-59.5939,77.68,72,77,2.33,broken clouds
10,10,Vaini,TO,-21.2000,-175.2000,77.16,65,40,13.80,scattered clouds
13,13,Carnarvon,AU,-24.8667,113.6333,71.67,75,0,11.50,clear sky
14,14,Katsuura,JP,35.1333,140.3000,71.71,89,100,13.38,overcast clouds
18,18,Tucumcari,US,35.1717,-103.7250,82.24,42,1,8.05,clear sky
19,19,Vostok,RU,46.4856,135.8833,69.21,63,14,2.33,few clouds
21,21,Dingle,PH,10.9995,122.6711,84.20,75,50,5.48,light rain
22,22,Westport,US,41.1415,-73.3579,75.34,74,1,1.99,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities.isnull().sum()

City_ID        0
City           0
Country        0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [6]:

# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities = preferred_cities.dropna() #there weren't any null values so nothing changed
preferred_cities.count()

City_ID        342
City           342
Country        342
Lat            342
Lng            342
Max Temp       342
Humidity       342
Cloudiness     342
Wind Speed     342
Description    342
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
4,Rikitea,PF,74.82,clear sky,-23.1203,-134.9692,
5,Lagos,NG,77.32,overcast clouds,6.5833,3.7500,
6,Borba,BR,77.68,broken clouds,-4.3878,-59.5939,
10,Vaini,TO,77.16,scattered clouds,-21.2000,-175.2000,
13,Carnarvon,AU,71.67,clear sky,-24.8667,113.6333,
14,Katsuura,JP,71.71,overcast clouds,35.1333,140.3000,
18,Tucumcari,US,82.24,clear sky,35.1717,-103.7250,
19,Vostok,RU,69.21,few clouds,46.4856,135.8833,
21,Dingle,PH,84.20,light rain,10.9995,122.6711,
22,Westport,US,75.34,clear sky,41.1415,-73.3579,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f'{lat},{lng}'
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('No hotel found... skipping!')

No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!


In [9]:
hotel_df.count()

City           342
Country        342
Max Temp       342
Description    342
Lat            342
Lng            342
Hotel Name     342
dtype: int64

In [20]:
# 7. Drop the rows where there is no Hotel Name. We should expect 15 rows to drop
hotel_df[hotel_df['Hotel Name'] == ''] #This code tells us there are 15 rows without hotel name
hotel_df = hotel_df[hotel_df['Hotel Name'] != '']
hotel_df.count() # Now we have 15 less rows by dropping the ones that didn't find any hotel

City           322
Country        322
Max Temp       322
Description    322
Lat            322
Lng            322
Hotel Name     322
dtype: int64

In [21]:
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
4,Rikitea,PF,74.82,clear sky,-23.1203,-134.9692,People ThankYou
6,Borba,BR,77.68,broken clouds,-4.3878,-59.5939,Hotel da Amazônia
10,Vaini,TO,77.16,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
13,Carnarvon,AU,71.67,clear sky,-24.8667,113.6333,Hospitality Carnarvon
14,Katsuura,JP,71.71,overcast clouds,35.1333,140.3000,Katsuura Hotel Mikazuki
18,Tucumcari,US,82.24,clear sky,35.1717,-103.7250,"Holiday Inn Express & Suites Tucumcari, an IHG..."
21,Dingle,PH,84.20,light rain,10.9995,122.6711,SEGAYA PROPERTY
22,Westport,US,75.34,clear sky,41.1415,-73.3579,The Westport Inn
24,Nargana,PA,80.19,overcast clouds,9.4428,-78.5906,Dubbir
27,Atuona,PF,77.97,clear sky,-9.8000,-139.0333,Villa Enata


In [22]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [25]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Description}</dd>
<dt>Max Temperature</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [26]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))